In [ ]:
import dask
import logging
from datetime import datetime

from grizly.scheduling.registry import Job

## Job

In [ ]:
@dask.delayed
def print_time():
    print(f"Hello, it's {datetime.now().__str__()}")
    
time_log = print_time()

@dask.delayed
def add(x, y):
    return x + y

_sum = add(1, 2)

In [ ]:
job1 = Job("test_print")
job1.register(tasks=[time_log], 
              crons="* * * * *", 
              if_exists="replace")

job2 = Job("test_sum")
job2.register(tasks=[_sum], 
              upstream={"test_print": "success"}, 
              if_exists="replace")

In [ ]:
job1.info()

In [ ]:
job1.runs

In [ ]:
job2.info()

In [ ]:
job2.runs

In [ ]:
job1.submit(to_dask=False)

In [ ]:
job1.last_run.info()

In [ ]:
job1.unregister(remove_job_runs=True)

In [ ]:
job2.unregister()